# Tutorial 1: Bayes con un estado binario oculto

---
# Setup


##  Instalar e importar gadget de realimentación


In [1]:
# @title Install and import feedback gadget

!pip3 install vibecheck datatops --quiet

from vibecheck import DatatopsContentReviewContainer
def content_review(notebook_section: str):
    return DatatopsContentReviewContainer(
        "",  # No text prompt
        notebook_section,
        {
            "url": "https://pmyvdlilci.execute-api.us-east-1.amazonaws.com/klab",
            "name": "neuromatch_cn",
            "user_key": "y1x3mpx5",
        },
    ).render()


feedback_prefix = "W3D1_T1"

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.5 MB/s eta 0:00:00


In [2]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches, transforms, gridspec
from scipy.optimize import fsolve
from collections import namedtuple

##  Configuración de figuras


In [3]:
# @title Figure Settings
import logging
logging.getLogger('matplotlib.font_manager').disabled = True

import ipywidgets as widgets  # interactive display
from ipywidgets import GridspecLayout, HBox, VBox, FloatSlider, Layout, ToggleButtons
from ipywidgets import interactive, interactive_output, Checkbox, Select
from IPython.display import clear_output
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/nma.mplstyle")

import warnings
warnings.filterwarnings("ignore")

## Funciones de graficado


In [4]:
# @title Plotting Functions
from matplotlib import colors

def plot_joint_probs(P, ):
  assert np.all(P >= 0), "probabilities should be >= 0"
  # normalize if not
  P = P / np.sum(P)
  marginal_y = np.sum(P,axis=1)
  marginal_x = np.sum(P,axis=0)

  # definitions for the axes
  left, width = 0.1, 0.65
  bottom, height = 0.1, 0.65
  spacing = 0.005

  # start with a square Figure
  fig = plt.figure(figsize=(5, 5))

  joint_prob = [left, bottom, width, height]
  rect_histx = [left, bottom + height + spacing, width, 0.2]
  rect_histy = [left + width + spacing, bottom, 0.2, height]

  rect_x_cmap = plt.cm.Blues
  rect_y_cmap = plt.cm.Reds

  # Show joint probs and marginals
  ax = fig.add_axes(joint_prob)
  ax_x = fig.add_axes(rect_histx, sharex=ax)
  ax_y = fig.add_axes(rect_histy, sharey=ax)

  # Show joint probs and marginals
  ax.matshow(P,vmin=0., vmax=1., cmap='Greys')
  ax_x.bar(0, marginal_x[0], facecolor=rect_x_cmap(marginal_x[0]))
  ax_x.bar(1, marginal_x[1], facecolor=rect_x_cmap(marginal_x[1]))
  ax_y.barh(0, marginal_y[0], facecolor=rect_y_cmap(marginal_y[0]))
  ax_y.barh(1, marginal_y[1], facecolor=rect_y_cmap(marginal_y[1]))
  # set limits
  ax_x.set_ylim([0, 1])
  ax_y.set_xlim([0, 1])

  # show values
  ind = np.arange(2)
  x,y = np.meshgrid(ind,ind)
  for i,j in zip(x.flatten(), y.flatten()):
    c = f"{P[i, j]:.2f}"
    ax.text(j,i, c, va='center', ha='center', color='black')
  for i in ind:
    v = marginal_x[i]
    c = f"{v:.2f}"
    ax_x.text(i, v +0.1, c, va='center', ha='center', color='black')
    v = marginal_y[i]
    c = f"{v:.2f}"
    ax_y.text(v+0.2, i, c, va='center', ha='center', color='black')

  # set up labels
  ax.xaxis.tick_bottom()
  ax.yaxis.tick_left()
  ax.set_xticks([0,1])
  ax.set_yticks([0,1])
  ax.set_xticklabels(['Silver', 'Gold'])
  ax.set_yticklabels(['Small', 'Large'])
  ax.set_xlabel('color')
  ax.set_ylabel('size')
  ax_x.axis('off')
  ax_y.axis('off')
  return fig


def plot_prior_likelihood_posterior(prior, likelihood, posterior):
  # definitions for the axes
  left, width = 0.05, 0.3
  bottom, height = 0.05, 0.9
  padding = 0.12
  small_width = 0.1
  left_space = left + small_width + padding
  added_space = padding + width

  fig = plt.figure(figsize=(12, 4))

  rect_prior = [left, bottom, small_width, height]
  rect_likelihood = [left_space , bottom , width, height]
  rect_posterior = [left_space +  added_space, bottom , width, height]

  ax_prior = fig.add_axes(rect_prior)
  ax_likelihood = fig.add_axes(rect_likelihood, sharey=ax_prior)
  ax_posterior = fig.add_axes(rect_posterior, sharey = ax_prior)

  rect_colormap = plt.cm.Blues

  # Show posterior probs and marginals
  ax_prior.barh(0, prior[0], facecolor=rect_colormap(prior[0, 0]))
  ax_prior.barh(1, prior[1], facecolor=rect_colormap(prior[1, 0]))
  ax_likelihood.matshow(likelihood, vmin=0., vmax=1., cmap='Reds')
  ax_posterior.matshow(posterior, vmin=0., vmax=1., cmap='Greens')

  # Probabilities plot details
  ax_prior.set(xlim=[1, 0], xticks=[], yticks=[0, 1],
                yticklabels=['left', 'right'], title="Prior p(s)")
  ax_prior.yaxis.tick_right()
  ax_prior.spines['left'].set_visible(False)
  ax_prior.spines['bottom'].set_visible(False)

  # Likelihood plot details
  ax_likelihood.set(xticks=[0, 1], xticklabels=['fish', 'no fish'],
                    yticks=[0, 1], yticklabels=['left', 'right'],
                    ylabel='state (s)', xlabel='measurement (m)',
                    title='Likelihood p(m (left) | s)')
  ax_likelihood.xaxis.set_ticks_position('bottom')
  ax_likelihood.spines['left'].set_visible(False)
  ax_likelihood.spines['bottom'].set_visible(False)

  # Posterior plot details
  ax_posterior.set(xticks=[0, 1], xticklabels=['fish', 'no fish'],
                    yticks=[0, 1], yticklabels=['left', 'right'],
                    ylabel='state (s)', xlabel='measurement (m)',
                    title='Posterior p(s | m)')
  ax_posterior.xaxis.set_ticks_position('bottom')
  ax_posterior.spines['left'].set_visible(False)
  ax_posterior.spines['bottom'].set_visible(False)

  # show values
  ind = np.arange(2)
  x,y = np.meshgrid(ind,ind)
  for i,j in zip(x.flatten(), y.flatten()):
    c = f"{posterior[i, j]:.2f}"
    ax_posterior.text(j, i, c, va='center', ha='center', color='black')
  for i,j in zip(x.flatten(), y.flatten()):
    c = f"{likelihood[i, j]:.2f}"
    ax_likelihood.text(j, i, c, va='center', ha='center', color='black')
  for i in ind:
    v = prior[i, 0]
    c = f"{v:.2f}"
    ax_prior.text(v+0.2, i, c, va='center', ha='center', color='black')


def plot_prior_likelihood(ps, p_a_s1, p_a_s0, measurement):
  likelihood = np.asarray([[p_a_s1, 1-p_a_s1], [p_a_s0, 1-p_a_s0]])
  assert 0.0 <= ps <= 1.0
  prior = np.asarray([ps, 1 - ps])
  if measurement == "Fish":
    posterior = likelihood[:, 0] * prior
  else:
    posterior = (likelihood[:, 1] * prior).reshape(-1)
  posterior /= np.sum(posterior)

  # definitions for the axes
  left, width = 0.05, 0.3
  bottom, height = 0.05, 0.9
  padding = 0.12
  small_width = 0.2
  left_space = left + small_width + padding
  small_padding = 0.05

  fig = plt.figure(figsize=(12, 4))

  rect_prior = [left, bottom, small_width, height]
  rect_likelihood = [left_space , bottom , width, height]
  rect_posterior = [left_space + width + small_padding, bottom , small_width, height]

  ax_prior = fig.add_axes(rect_prior)
  ax_likelihood = fig.add_axes(rect_likelihood, sharey=ax_prior)
  ax_posterior = fig.add_axes(rect_posterior, sharey=ax_prior)

  prior_colormap = plt.cm.Blues
  posterior_colormap = plt.cm.Greens

  # Show posterior probs and marginals
  ax_prior.barh(0, prior[0], facecolor=prior_colormap(prior[0]))
  ax_prior.barh(1, prior[1], facecolor=prior_colormap(prior[1]))
  ax_likelihood.matshow(likelihood, vmin=0., vmax=1., cmap='Reds')
  ax_posterior.barh(0, posterior[0], facecolor=posterior_colormap(posterior[0]))
  ax_posterior.barh(1, posterior[1], facecolor=posterior_colormap(posterior[1]))

  # Probabilities plot details
  ax_prior.set(xlim=[1, 0], yticks=[0, 1], yticklabels=['left', 'right'],
                title="Prior p(s)", xticks=[])
  ax_prior.yaxis.tick_right()
  ax_prior.spines['left'].set_visible(False)
  ax_prior.spines['bottom'].set_visible(False)

  # Likelihood plot details
  ax_likelihood.set(xticks=[0, 1], xticklabels=['fish', 'no fish'],
                    yticks=[0, 1], yticklabels=['left', 'right'],
                    ylabel='state (s)', xlabel='measurement (m)',
                    title='Likelihood p(m | s)')
  ax_likelihood.xaxis.set_ticks_position('bottom')
  ax_likelihood.spines['left'].set_visible(False)
  ax_likelihood.spines['bottom'].set_visible(False)

  # Posterior plot details
  ax_posterior.set(xlim=[0, 1], xticks=[], yticks=[0, 1],
                    yticklabels=['left', 'right'],
                    title="Posterior p(s | m)")
  ax_posterior.spines['left'].set_visible(False)
  ax_posterior.spines['bottom'].set_visible(False)

  # show values
  ind = np.arange(2)
  x,y = np.meshgrid(ind, ind)
  for i in ind:
    v = posterior[i]
    c = f"{v:.2f}"
    ax_posterior.text(v+0.2, i, c, va='center', ha='center', color='black')
  for i,j in zip(x.flatten(), y.flatten()):
    c = f"{likelihood[i, j]:.2f}"
    ax_likelihood.text(j, i, c, va='center', ha='center', color='black')
  for i in ind:
    v = prior[i]
    c = f"{v:.2f}"
    ax_prior.text(v+0.2, i, c, va='center', ha='center', color='black')
  plt.show()
  return fig


def plot_utility(ps):
  prior = np.asarray([ps, 1 - ps])

  utility = np.array([[2, -3], [-2, 1]])

  expected = prior @ utility

  # definitions for the axes
  left, width = 0.05, 0.16
  bottom, height = 0.05, 0.9
  padding = 0.02
  small_width = 0.1
  left_space = left + small_width + padding
  added_space = padding + width

  fig = plt.figure(figsize=(17, 3))

  rect_prior = [left, bottom, small_width, height]
  rect_utility = [left + added_space , bottom , width, height]
  rect_expected = [left + 2* added_space, bottom , width, height]

  ax_prior = fig.add_axes(rect_prior)
  ax_utility = fig.add_axes(rect_utility, sharey=ax_prior)
  ax_expected = fig.add_axes(rect_expected)

  rect_colormap = plt.cm.Blues

  # Data of plots
  ax_prior.barh(0, prior[0], facecolor=rect_colormap(prior[0]))
  ax_prior.barh(1, prior[1], facecolor=rect_colormap(prior[1]))
  ax_utility.matshow(utility, cmap='cool')
  norm = colors.Normalize(vmin=-3, vmax=3)
  ax_expected.bar(0, expected[0], facecolor=rect_colormap(norm(expected[0])))
  ax_expected.bar(1, expected[1], facecolor=rect_colormap(norm(expected[1])))

  # Probabilities plot details
  ax_prior.set(xlim=[1, 0], xticks=[], yticks=[0, 1],
                yticklabels=['left', 'right'], title="Probability of state")
  ax_prior.yaxis.tick_right()
  ax_prior.spines['left'].set_visible(False)
  ax_prior.spines['bottom'].set_visible(False)

  # Utility plot details
  ax_utility.set(xticks=[0, 1], xticklabels=['left', 'right'],
                  yticks=[0, 1], yticklabels=['left', 'right'],
                  ylabel='state (s)', xlabel='action (a)',
                  title='Utility')
  ax_utility.xaxis.set_ticks_position('bottom')
  ax_utility.spines['left'].set_visible(False)
  ax_utility.spines['bottom'].set_visible(False)

  # Expected utility plot details
  ax_expected.set(title='Expected utility', ylim=[-3, 3],
                  xticks=[0, 1], xticklabels=['left', 'right'],
                  xlabel='action (a)', yticks=[])
  ax_expected.xaxis.set_ticks_position('bottom')
  ax_expected.spines['left'].set_visible(False)
  ax_expected.spines['bottom'].set_visible(False)

  # show values
  ind = np.arange(2)
  x,y = np.meshgrid(ind,ind)

  for i,j in zip(x.flatten(), y.flatten()):
    c = f"{utility[i, j]:.2f}"
    ax_utility.text(j, i, c, va='center', ha='center', color='black')
  for i in ind:
    v = prior[i]
    c = f"{v:.2f}"
    ax_prior.text(v+0.2, i, c, va='center', ha='center', color='black')
  for i in ind:
    v = expected[i]
    c = f"{v:.2f}"
    ax_expected.text(i, 2.5, c, va='center', ha='center', color='black')

  return fig


def plot_prior_likelihood_utility(ps, p_a_s1, p_a_s0, measurement):
  assert 0.0 <= ps <= 1.0
  assert 0.0 <= p_a_s1 <= 1.0
  assert 0.0 <= p_a_s0 <= 1.0
  prior = np.asarray([ps, 1 - ps])
  likelihood = np.asarray([[p_a_s1, 1-p_a_s1],[p_a_s0, 1-p_a_s0]])
  utility = np.array([[2.0, -3.0], [-2.0, 1.0]])

  if measurement == "Fish":
    posterior = likelihood[:, 0] * prior
  else:
    posterior = (likelihood[:, 1] * prior).reshape(-1)
  posterior /= np.sum(posterior)
  expected = posterior @ utility

  # definitions for the axes
  left, width = 0.05, 0.3
  bottom, height = 0.05, 0.3
  padding = 0.12
  small_width = 0.2
  left_space = left + small_width + padding
  small_padding = 0.05

  fig = plt.figure(figsize=(10, 9))

  rect_prior = [left, bottom + height + padding, small_width, height]
  rect_likelihood = [left_space , bottom + height + padding , width, height]
  rect_posterior = [left_space + width + small_padding,
                    bottom + height + padding,
                    small_width, height]

  rect_utility = [padding, bottom, width, height]
  rect_expected = [padding + width + padding + left, bottom, width, height]

  ax_likelihood = fig.add_axes(rect_likelihood)
  ax_prior = fig.add_axes(rect_prior, sharey=ax_likelihood)
  ax_posterior = fig.add_axes(rect_posterior, sharey=ax_likelihood)
  ax_utility = fig.add_axes(rect_utility)
  ax_expected = fig.add_axes(rect_expected)

  prior_colormap = plt.cm.Blues
  posterior_colormap = plt.cm.Greens
  expected_colormap = plt.cm.Wistia

  # Show posterior probs and marginals
  ax_prior.barh(0, prior[0], facecolor=prior_colormap(prior[0]))
  ax_prior.barh(1, prior[1], facecolor=prior_colormap(prior[1]))
  ax_likelihood.matshow(likelihood, vmin=0., vmax=1., cmap='Reds')
  ax_posterior.barh(0, posterior[0], facecolor=posterior_colormap(posterior[0]))
  ax_posterior.barh(1, posterior[1], facecolor=posterior_colormap(posterior[1]))
  ax_utility.matshow(utility, vmin=0., vmax=1., cmap='cool')
  ax_expected.bar(0, expected[0], facecolor=expected_colormap(expected[0]))
  ax_expected.bar(1, expected[1], facecolor=expected_colormap(expected[1]))

  # Probabilities plot details
  ax_prior.set(xlim=[1, 0], yticks=[0, 1], yticklabels=['left', 'right'],
                title="Prior p(s)", xticks=[])
  ax_prior.yaxis.tick_right()
  ax_prior.spines['left'].set_visible(False)
  ax_prior.spines['bottom'].set_visible(False)

  # Likelihood plot details
  ax_likelihood.set(xticks=[0, 1], xticklabels=['fish', 'no fish'],
                    yticks=[0, 1], yticklabels=['left', 'right'],
                    ylabel='state (s)', xlabel='measurement (m)',
                    title='Likelihood p(m | s)')
  ax_likelihood.xaxis.set_ticks_position('bottom')
  ax_likelihood.spines['left'].set_visible(False)
  ax_likelihood.spines['bottom'].set_visible(False)

  # Posterior plot details
  ax_posterior.set(xlim=[0, 1], xticks=[], yticks=[0, 1],
                    yticklabels=['left', 'right'],
                    title="Posterior p(s | m)")
  ax_posterior.spines['left'].set_visible(False)
  ax_posterior.spines['bottom'].set_visible(False)

  # Utility plot details
  ax_utility.set(xticks=[0, 1], xticklabels=['left', 'right'],
                  xlabel='action (a)', yticks=[0, 1],
                  yticklabels=['left', 'right'],
                  title='Utility', ylabel='state (s)')
  ax_utility.xaxis.set_ticks_position('bottom')
  ax_utility.spines['left'].set_visible(False)
  ax_utility.spines['bottom'].set_visible(False)

  # Expected Utility plot details
  ax_expected.set(ylim=[-2, 2], xticks=[0, 1],
                  xticklabels=['left', 'right'],
                  xlabel='action (a)',
                  title='Expected utility', yticks=[])
  ax_expected.spines['left'].set_visible(False)

  # show values
  ind = np.arange(2)
  x,y = np.meshgrid(ind,ind)
  for i in ind:
    v = posterior[i]
    c = f"{v:.2f}"
    ax_posterior.text(v+0.2, i, c, va='center', ha='center', color='black')
  for i,j in zip(x.flatten(), y.flatten()):
    c = f"{likelihood[i,j]:.2f}"
    ax_likelihood.text(j, i, c, va='center', ha='center', color='black')
  for i,j in zip(x.flatten(), y.flatten()):
    c = f"{utility[i, j]:.2f}"
    ax_utility.text(j,i, c, va='center', ha='center', color='black')
  for i in ind:
    v = prior[i]
    c = f"{v:.2f}"
    ax_prior.text(v+0.2, i, c, va='center', ha='center', color='black')
  for i in ind:
    v = expected[i]
    c = f"{v:.2f}"
    ax_expected.text(i, v, c, va='center', ha='center', color='black')
  plt.show()
  return fig

##  Funciones de ayuda


In [5]:
# @title Helper Functions

def compute_marginal(px, py, cor):
  """ Calculate 2x2 joint probabilities given marginals p(x=1), p(y=1) and correlation

    Args:
      px (scalar): marginal probability of x
      py (scalar): marginal probability of y
      cor (scalar): correlation value

    Returns:
      ndarray of size (2, 2): joint probability array of x and y
  """

  p11 = px*py + cor*np.sqrt(px*py*(1-px)*(1-py))
  p01 = px - p11
  p10 = py - p11
  p00 = 1.0 - p11 - p01 - p10

  return np.asarray([[p00, p01], [p10, p11]])


def compute_cor_range(px,py):
  """ Calculate the allowed range of correlation values given marginals p(x=1)
    and p(y=1)

  Args:
    px (scalar): marginal probability of x
    py (scalar): marginal probability of y

  Returns:
    scalar, scalar: minimum and maximum possible values of correlation
  """

  def p11(corr):
    return px*py + corr*np.sqrt(px*py*(1-px)*(1-py))
  def p01(corr):
    return px - p11(corr)
  def p10(corr):
    return py - p11(corr)
  def p00(corr):
    return 1.0 - p11(corr) - p01(corr) - p10(corr)
  Cmax = min(fsolve(p01, 0.0), fsolve(p10, 0.0))
  Cmin = max(fsolve(p11, 0.0), fsolve(p00, 0.0))
  return Cmin, Cmax

---
# Sección 0: Introducción

---
# Sección 1: Nos fuimo' a pescar (Gone Fishin')


---
# Sección 2: Decidiendo dónde pescar

## Demo Interactivo 2: Explorando la decisión

### Preguntas

1. Acabas de llegar al muelle por primera vez y no tienes idea de dónde podría estar el pez. Entonces supones que la probabilidad de que la escuela esté en el lado izquierdo es 0,5 (por lo que la probabilidad de que esté en el lado derecho también es 0,5). ¿De qué lado elegirías pescar dados nuestros valores de utilidad?
2. Piensas que la probabilidad de que la escuela esté en el lado izquierdo es muy baja (0,1) y correspondientemente alta en el lado derecho (0,9). ¿De qué lado elegirías pescar dados nuestros valores de utilidad?
3. ¿Qué elegirías si la probabilidad de que la escuela esté en el lado izquierdo sea ligeramente menor que en el lado derecho (0,4 frente a 0,6)?

Ejecute esta celda para usar el widget

In [6]:
# @markdown Execute this cell to use the widget
ps_widget = widgets.FloatSlider(0.9, description='p(s = left)',
                                min=0.0, max=1.0, step=0.01)

@widgets.interact(
    ps = ps_widget,
)
def make_utility_plot(ps):
  fig = plot_utility(ps)
  plt.show(fig)
  plt.close(fig)

interactive(children=(FloatSlider(value=0.9, description='p(s = left)', max=1.0, step=0.01), Output()), _dom_c…

---
# Sección 3: Probabilidad de que los peces estén en cada lado

## Piensa! 3: Adivinando la ubicación del pez

Digamos que vamos a pescar a otro muelle. Aquí, existen diferentes probabilidades de pescar dado el estado del mundo. En este muelle, si pescas en el lado del muelle donde están los peces, tienes un 70% de posibilidades de pescar. Si pescas en el lado equivocado, pescarás con sólo un 20% de probabilidad. ¡Éstas son las verosimilitudes de observar a alguien pescando un pez! Es decir, ¡estás tomando una medida viendo si alguien más pesca un pez!

Ves que un pescador está pescando en el lado izquierdo.

1) Averigüe cada uno de los siguientes (puede ser más fácil hacerlo por separado y luego comparar resultados):
- probabilidad de pescar un pez dado que el banco de peces está en el lado izquierdo, $P(m = \textrm{capturar pez} | s = \textrm{izquierda} )$
- probabilidad de no capturar un pez dado que el banco de peces está en el lado izquierdo, $P(m = \textrm{no captura pez} | s = \textrm{izquierda})$
- probabilidad de pescar un pez dado que el banco de peces está en el lado derecho, $P(m = \textrm{capturar pez} | s = \textrm{derecha})$
- probabilidad de no capturar un pez dado que el banco de peces está en el lado derecho, $P(m = \textrm{no captura pez} | s = \textrm{derecha})$

2) Si el pescador pesca un pez, ¿de qué lado creerías que está el banco? ¿Por qué?

3) Si el pescador no pesca ningún pez, ¿de qué lado crees que está el banco? ¿Por qué?

En el ejercicio anterior, intentaste adivinar dónde estaba el banco de peces basándose en la medición que tomaste (observar a alguien pescar). Lo hiciste eligiendo el estado (lado donde crees que están los peces) que maximiza la probabilidad de la medición. En otras palabras, estimó el estado maximizando la verosimilitud (el lado con la mayor probabilidad de medición dado el estado $P(m|s$)). Esto se llama estimación de máxima verosimilitud (MLE, por _maximum likelihood estimation_) y lo ha encontrado antes durante este curso, en el [día de estadísticas de los pre-requisitos] (https://compneuro.neuromatch.io/tutorials/W0D5_Statistics/student/W0D5_Tutorial2.html# sección-2-2-máxima-verosimilitud) y el [Día de ajustado de modelos](https://compneuro.neuromatch.io/tutorials/W1D3_ModelFitting/student/W1D3_Tutorial2.html)!

Pero, ¿y si llevaras años yendo a este muelle y supieras que los peces casi siempre están en el lado izquierdo? Esto probablemente debería afectar la forma en que realiza su estimación: confiaría menos en la nueva medición única y más en su conocimiento previo. ¡Esta es la idea fundamental detrás de la inferencia bayesiana, como veremos más adelante en este tutorial!

---
# Sección 4: Correlación y marginación

## Section 4.1: Correlación

En esta sección, daremos un paso atrás un poco y pensaremos de manera más general en la cantidad de información compartida entre dos variables aleatorias. Queremos saber cuánta información obtienes cuando observas una variable (tomar una medición) si sabes algo sobre otra. Veremos que el concepto fundamental es el mismo si pensamos en dos atributos, por ejemplo el tamaño y color del pez, o la información previa y la verosimilitud.

### ¡Pensar! 4.1: Distribuciones de probabilidad covariantes

La relación entre las probabilidades marginales y las probabilidades conjuntas está determinada por la correlación entre las dos variables aleatorias, una medida normalizada de cuánto covarían las variables. También podemos pensar en esto como ganar información sobre una de las variables cuando observamos una medición de la otra. Pensaremos en esto de manera más formal en el Tutorial 2.

Aquí queremos pensar en cómo la correlación entre el tamaño y el color de estos peces cambia la cantidad de información que obtenemos sobre un atributo en función del otro. Consulte la Sección Bonus 1 para conocer la fórmula de correlación.

Utilice el widget a continuación y responda las siguientes preguntas:

1. Cuando la correlación es cero, $\rho = 0$, ¿qué te dice la distribución de tamaño sobre el color?
2. Establezca $\rho$ en algo pequeño. Al cambiar la probabilidad de que aparezcan peces dorados, ¿qué sucede con la proporción de probabilidades de tamaño? Establezca $\rho$ más grande (puede ser negativo). ¿Puedes explicar el patrón de cambios en las probabilidades de tamaño a medida que cambias la probabilidad del pez dorado?
3. Establezca la probabilidad de peces dorados y peces grandes en alrededor del 65%. A medida que la correlación tiende a 1, ¿con qué frecuencia verá peces grandes plateados?
4. ¿Qué te dice el aumento de la correlación (absoluta) sobre la probabilidad de que veas una de las propiedades si ves un pez junto con la otra propiedad?

Ejecute esta celda para habilitar el widget

In [7]:
# @markdown Execute this cell to enable the widget
style = {'description_width': 'initial'}
gs = GridspecLayout(2,2)

cor_widget = widgets.FloatSlider(0.0, description='ρ',
                                 min=-1, max=1, step=0.01)
px_widget = widgets.FloatSlider(0.5, description='p(color=golden)',
                                min=0.01, max=0.99, step=0.01,
                                style=style)
py_widget = widgets.FloatSlider(0.5, description='p(size=large)',
                                min=0.01, max=0.99, step=0.01,
                                style=style)
gs[0, 0] = cor_widget
gs[0, 1] = px_widget
gs[1, 0] = py_widget


@widgets.interact(
    px=px_widget,
    py=py_widget,
    cor=cor_widget,
)


def make_corr_plot(px, py, cor):
  Cmin, Cmax = compute_cor_range(px, py)  # allow correlation values
  cor_widget.min, cor_widget.max = Cmin+0.01, Cmax-0.01
  if cor_widget.value > Cmax:
    cor_widget.value = Cmax
  if cor_widget.value < Cmin:
    cor_widget.value = Cmin
  cor = cor_widget.value
  P = compute_marginal(px,py,cor)
  fig = plot_joint_probs(P)
  plt.show(fig)
  plt.close(fig)

interactive(children=(FloatSlider(value=0.5, description='p(color=golden)', layout=Layout(grid_area='widget002…

Acabamos de ver cómo dos variables aleatorias pueden ser más o menos independientes. Cuanto más correlacionada, menos independiente y más compartida es la información. También aprendimos que podemos marginalizar para determinar la probabilidad marginal de una medición o para encontrar la distribución de probabilidad marginal de dos variables aleatorias. ¡Ahora vamos a completar nuestro viaje hacia ser completamente bayesianos!

### Ejercicio de matemáticas 4.2.2: Calcular la verosimilitud marginal

Cuando normalizamos para encontrar la probabilidad posterior, necesitamos determinar la verosimilitud marginal (o evidencia) de la medición que observamos. Para hacer esto, necesitamos marginar como acabamos de hacer arriba para encontrar las probabilidades de un color o tamaño. ¡Solo que en este caso estamos marginando para eliminar una variable condicionante! En este caso, consideremos la probabilidad (verosimilitud) de que haya peces (si observamos a un pescador pescando a la **derecha**).


| p(m\|s)       | m = pez | m = no pez  |
| ------------ | ---------- | -------------- |
| s = izquierda     | 0.1      | 0.9          |
| s = derecha    | 0.5      | 0.5          |


La tabla anterior nos muestra las **verosimilitudes**, tal como exploramos anteriormente.

Quieres saber la probabilidad total de que un pez sea capturado, $P(m = \textrm{pez})$, por el pescador que pesca a la derecha. (Necesitaría esto para calcular el posterior). Para hacer esto, deberá considerar la probabilidad previa, $p(s)$, y marginar sobre los estados ocultos!

Este es un ejemplo de cómo marginar o condicionar también la variable que no nos interesa.

Complete los siguientes problemas matemáticos para practicar más el pensamiento mediante probabilidades:

1. Calcule la probabilidad marginal de que se capture el pez, $P(m = \textrm{pez})$, si los factores anteriores son: $p(s = \textrm{izquierda}) = 0.3$ y $p(s = \textrm{derecha}) = 0,7$.
2. Calcule la probabilidad marginal de que se capture el pez, $P(m = \textrm{pez})$, si los factores anteriores son: $p(s = \textrm{izquierda}) = 0.6$ y $p(s = \textrm{derecha}) = 0,4$.

---
# Sección 5: Regla de Bayes y la probabilidad posterior

La marginalización se utilizará para combinar nuestro conocimiento previo, al que llamamos **probabilidad a priori** (_prior_), y nuestra nueva información de una medición, la **verosimilitud** (_likelihood_). Sólo que en este caso, la información que obtenemos sobre el estado oculto que nos interesa, donde se encuentran los peces, se basa en la relación entre las probabilidades de la medición y nuestra probabilidad a priori.

Ahora podemos calcular la distribución posterior completa para el estado oculto ($s$) usando la regla de Bayes. Como hemos visto, el posterior es proporcional a la probabilidad a priori multiplicado por la verosimilitud. Esto significa que la probabilidad posterior del estado oculto ($s$) dada una medida ($m$) es proporcional a la probabilidad de la medición dado el estado multiplicada por la probabilidad a priori de ese estado:

\begin{equation}
P(s | m) \propto P(m | s) P(s)
\end{equation}

Decimos proporcional a en lugar de igual porque necesitamos normalizar para producir una distribución de probabilidad completa:

\begin{equation}
P(s | m) = \frac{P(m | s) P(s)}{P(m)}
\end{equation}

Normalizar con este $P(m)$ significa que nuestro posterior es una distribución de probabilidad completa que suma o integra a 1 apropiadamente. ¡Ahora podemos usar esta nueva y completa distribución de probabilidad para cualquier inferencia o decisión futura que queramos! De hecho, como veremos mañana, ¡podremos usarlo como nueva probabilidad a priori! Finalmente, a menudo llamamos a esta distribución de probabilidad nuestras creencias sobre los estados ocultos, para enfatizar que es nuestro conocimiento subjetivo sobre el estado oculto.

Para muchos casos complicados, como los que podríamos estar usando para modelar inferencias conductuales o cerebrales, el término de normalización puede ser intratable o extremadamente complejo de calcular. Podemos tener cuidado al elegir distribuciones de probabilidad donde podamos calcular analíticamente la probabilidad posterior o donde la aproximación numérica sea confiable. Mejor aún, ¡a veces no necesitamos preocuparnos por esta normalización! El término de normalización, $P(m)$, es la probabilidad de la medición. Esto no depende del estado, por lo que es esencialmente una constante que a menudo podemos ignorar. Podemos comparar los valores de distribución posterior no normalizados para diferentes estados porque la forma en que se relacionan entre sí no cambia cuando se dividen por la misma constante. Mañana veremos cómo hacer esto para comparar evidencia de diferentes hipótesis. (También se utiliza para comparar la probabilidad de los modelos ajustados utilizando la estimación de máxima verosimilitud)

En este ejemplo relativamente simple, podemos calcular la probabilidad marginal $P(m)$ fácilmente usando:

\begin{equation}
P(m) = \sum_s P(m | s) P(s)
\end{equation}

Luego podemos normalizar para tratar con la distribución posterior completa.

## Ejercicio matemático 5: Calcular una probabilidad posterior

Nuestra probabilidad a priori es $p(s = \textrm{izquierda}) = 0.3$ y $p(s = \textrm{derecha}) = 0.7$. En el video, aprendimos que la probabilidad de pescar un pez si pescaban del mismo lado que el banco era del 50%. En caso contrario, era el 10%. Observamos a una persona pescando por el lado izquierdo. Nuestra probabilidad es:


| Verosimilitud: p(m \| s) | m = pez   | m = no pez  |
| ----------------- |----------|----------|
| s = izquierda          | 0.5          | 0.5         |
| s = derecha         | 0.1        |  0.9       |


Calcule la probabilidad posterior (en papel) de que:

1. El banco está a la izquierda si el pescador pesca un pez: $p(s = \textrm{izquierda} | m = \textrm{pez})$ (pista: normalizar calculando $p(m = \textrm {pescado})$)
2. El banco está a la derecha si el pescador no pesca ningún pez: $p(s = \textrm{derecha} | m = \textrm{no pez})$

<br>

### Respuesta

Usando Bayes, sabemos que P(s = izquierda | m = pescado) = P(m = pescado | s = izquierda)P(s = izquierda) / P(m = pescado)

   Primero computemos P(m = pescado):

   P(m = pescado) =  P(m = pescado | s = izquierda)P(s = izquierda) +  P(m = pescado | s = derecha)P(s = derecha)
               = 0.5 * 0.3 + .1 * .7
               = 0.22

   Ahora lo metemos todo en Bayes:

   P(s = izquierda | m = pescado) = P(m = pescado | s = izquierda)P(s = izquierda) / P(m = pescado)
                          = 0.5 * 0.3 / 0.22
                          = 0.68

## Ejercicio de programación 5: Computando posteriores

Implementemos nuestra matemática anterior para poder calcular posteriores para diferentes probabilidades a priori y verosimilitudes.

Como antes, nuestro probabilidad a priori es $p(s = \textrm{izquierda}) = 0.3$ y $p(s = \textrm{derecha}) = 0.7$. En el video, aprendimos que la probabilidad de pescar un pez si pescaban del mismo lado que el banco era del 50%. En caso contrario, era el 10%. Observamos a una persona pescando por el lado izquierdo. Nuestra probabilidad es:


| Verosimilitud: p(m \| s) | m = pez   | m = no pez  |
| ----------------- |----------|----------|
| s = izquierda          | 0.5          | 0.5         |
| s = derecha         | 0.1        |  0.9       |


Queremos que nuestro posterior completo adopte la misma forma de 2 por 2. ¡Asegúrate de que los resultados coincidan con tus respuestas calculadas!


In [8]:
def compute_posterior(likelihood, prior):
  """ Use Bayes' Rule to compute posterior from likelihood and prior

  Args:
    likelihood (ndarray): i x j array with likelihood probabilities where i is
                    number of state options, j is number of measurement options
    prior (ndarray): i x 1 array with prior probability of each state

  Returns:
    ndarray: i x j array with posterior probabilities where i is
            number of state options, j is number of measurement options

  """

  # Compute unnormalized posterior (likelihood times prior)
  posterior = likelihood * prior # first row is s = left, second row is s = right

  # Compute p(m)
  p_m = np.sum(posterior, axis = 0)

  # Normalize posterior (divide elements by p_m)
  posterior /= p_m

  return posterior


# Make prior
prior = np.array([0.3, 0.7]).reshape((2, 1)) # first row is s = left, second row is s = right

# Make likelihood
likelihood = np.array([[0.5, 0.5], [0.1, 0.9]]) # first row is s = left, second row is s = right

# Compute posterior
posterior = compute_posterior(likelihood, prior)

# Visualize
# plot_prior_likelihood_posterior(prior, likelihood, posterior)

## Demo Interactivo 5: Que afecta al posterior?

Ahora que podemos entender la implementación de la *regla de Bayes*, varíemos los parámetros de la probabilidad a priori y la verosimilitud para ver cómo cambiarlos afectan al posterior.

En la demo a continuación, puedes cambiar la probabilidad a priori jugando con el control deslizante de $p( s = izquierda)$. También puedes cambiar la probabilidad cambiando la probabilidad de pescar un pez dado que el banco está a la izquierda y la probabilidad de pescar un pez dado que el banco está a la derecha. El pescador que estás observando está pescando a la izquierda.


1. Manteniendo constante la verosimilitud, ¿cuándo es que la probabilidad a priori tiene la mayor influencia sobre lo posterior? Es decir, ¿cuándo se parece más la parte posterior a la probabilidad a priori, sin importar si se pescó un pez o no?
2. ¿Qué sucede si las probabilidades de capturar un pez son similares cuando se pesca en el lado correcto o incorrecto?
3. Establezca la probabilidad previa del estado = izquierda en 0,6 y juegue con la verosimilitud. ¿Cuándo la verosimilitud ejerce mayor influencia sobre la parte posterior? (es decir, cuando es que los datos "pesan más"?)

Ejecute esta celda para habilitar el widget

In [9]:
# @markdown Execute this cell to enable the widget
# style = {'description_width': 'initial'}
ps_widget = widgets.FloatSlider(0.3, description='p(s = left)',
                                min=0.01, max=0.99, step=0.01)
p_a_s1_widget = widgets.FloatSlider(0.5,
                                    description='p(fish on left | state = left)',
                                    min=0.01, max=0.99, step=0.01, style=style,
                                    layout=Layout(width='370px'))
p_a_s0_widget = widgets.FloatSlider(0.1,
                                    description='p(fish on left | state = right)',
                                    min=0.01, max=0.99, step=0.01, style=style,
                                    layout=Layout(width='370px'))

observed_widget = ToggleButtons(options=['Fish', 'No Fish'],
    description='Observation (m) on the left:', disabled=False, button_style='',
    layout=Layout(width='auto', display="flex"),
    style={'description_width': 'initial'}
)

widget_ui = VBox([ps_widget,
                  HBox([p_a_s1_widget, p_a_s0_widget]),
                  observed_widget])
widget_out = interactive_output(plot_prior_likelihood,
                                {'ps': ps_widget,
                                'p_a_s1': p_a_s1_widget,
                                'p_a_s0': p_a_s0_widget,
                                'measurement': observed_widget})
display(widget_ui, widget_out)

Output()

# Sección 6: Tomar decisiones de pesca bayesianas

Exploraremos cómo considerar la utilidad esperada de una acción en función de nuestra creencia (la distribución posterior) sobre dónde creemos que están los peces. Ahora tenemos todos los componentes de una decisión bayesiana: nuestra información previa, la probabilidad dada una medida, la distribución posterior (creencia) y nuestra utilidad (las ganancias y pérdidas). ¡Esto nos permite considerar la relación entre el verdadero valor del estado oculto, $s$, y lo que *esperamos* obtener si actuamos, $a$, en base a nuestra creencia!

Usemos el siguiente widget para pensar en la relación entre estas distribuciones de probabilidad y la función de utilidad.

## Demo Interactivo 6: ¿Qué son más importantes, las probabilidades o las utilidades?

Ahora vamos a reunir todo lo que hemos aprendido para obtener algunas intuiciones sobre cómo se combinan cada uno de los elementos que intervienen en una decisión bayesiana. Recuerde, la suposición común en neurociencia, psicología, economía, ecología, etc. es que nosotros (humanos y animales) estamos tratando de maximizar nuestra utilidad esperada. Están sucediendo muchas cosas en esta demo, ya que reúne todo lo contenido en este tutorial en un solo lugar; dedique tiempo a asegurarse de comprender los controles y las gráficos, especialmente cómo se relaciona todo.

1. ¿Puedes encontrar una situación en la que la utilidad esperada sea la misma para ambas acciones?
2. ¿Qué es más importante para determinar la utilidad esperada: la probabilidad a priori o la medición nueva (la verosimilitud)?
3. ¿Por qué es éste un modelo normativo?
4. ¿Puedes pensar en formas en las que sería necesario ampliar este modelo para describir el comportamiento humano o animal?

### Utilidad esperada de tomar una acción ahora es:
## $ E[U(a)] = \sum_s U(s,a)p(s|m) $

Utilidades iguales:

p(left) = 0.3, p(m=fish|s=left) = 0.46, p(m=fish|s=right)=0.33, fish

p(left) = 0.6, p(m=fish|s=left) = 0.63, p(m=fish|s=right)=0.08, no fish


In [10]:
# @markdown Execute this cell to enable the widget
# style = {'description_width': 'initial'}

ps_widget = widgets.FloatSlider(0.3, description='p(s = left)',
                                min=0.01, max=0.99, step=0.01,
                                layout=Layout(width='300px'))
p_a_s1_widget = widgets.FloatSlider(0.5,
                                    description='p(fish on left | state = left)',
                                    min=0.01, max=0.99, step=0.01,
                                    style=style, layout=Layout(width='370px'))
p_a_s0_widget = widgets.FloatSlider(0.1,
                                    description='p(fish on left | state = right)',
                                    min=0.01, max=0.99, step=0.01,
                                    style=style, layout=Layout(width='370px'))

observed_widget = ToggleButtons(options=['Fish', 'No Fish'],
    description='Observation (m) on the left:', disabled=False, button_style='',
    layout=Layout(width='auto', display="flex"),
    style={'description_width': 'initial'}
)

widget_ui = VBox([ps_widget,
                  HBox([p_a_s1_widget, p_a_s0_widget]),
                  observed_widget])

widget_out = interactive_output(plot_prior_likelihood_utility,
                                {'ps': ps_widget,
                                'p_a_s1': p_a_s1_widget,
                                'p_a_s0': p_a_s0_widget,
                                'measurement': observed_widget})
display(widget_ui, widget_out)

Output()